In [1]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np
from io import StringIO
import json

In [4]:
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

def parse_html_table(html):
    """Parse one HTML table into DataFrame with Date, Subject, Remarks, Link."""
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        return None

    rows = []
    for tr in table.select("tbody tr"):
        cols = tr.find_all("td")
        if len(cols) < 3:
            print("There are less than three cols, skipping")
            continue

        date = cols[0].get_text(strip=True)
        remarks = cols[2].get_text(strip=True)

        a = cols[1].find("a")
        subject = a.get_text(" ", strip=True) if a else cols[1].get_text(strip=True)
        link = ""
        if a and a.get("onclick"):
            onclick = a["onclick"]
            if "newwindow1" in onclick:
                start = onclick.find("'") + 1
                end = onclick.rfind("'")
                link = onclick[start:end]

        rows.append([date, subject, remarks, link])

    return pd.DataFrame(rows, columns=["Date", "Subject", "Remarks", "Link"])

def clean_sheet_name(name, fallback="Sheet"):
    safe = re.sub(r'[\[\]\:\*\?\/\\]', "_", str(name))
    return safe[:31] if safe else fallback

def cache_to_excel(cache_file, excel_out="Orders_All.xlsx"):
    with open(cache_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    used_names = {}  # track counts per sheet name

    with pd.ExcelWriter(excel_out, engine="xlsxwriter") as writer:
        for scraped in data["records"][0]["scraped_data"]:
            if not scraped.get("data_present"):
                continue

            action = scraped.get("action", "unknown")
            webpage = scraped.get("webpage", "")
            for resp in scraped.get("response", []):
                if resp.get("type") != "table_html":
                    continue

                df = parse_html_table(resp["value"])
                if df is None or df.empty:
                    continue

                titles = resp.get("title", [])
                base_name = clean_sheet_name(titles[-1] if titles else action)

                count = used_names.get(base_name, 0) + 1
                used_names[base_name] = count
                sheet_name = base_name if count == 1 else clean_sheet_name(f"{base_name}_{count}")


                meta_rows = [["Action", action], ["Webpage", webpage]]
                for i, t in enumerate(titles, start=1):
                    meta_rows.append([f"Header{i}", t])
                meta = pd.DataFrame(meta_rows, columns=["Meta", "Value"])
                meta.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

                df_visible = df[["Date", "Subject", "Remarks","Link"]]
                startrow = len(meta) + 2
                df_visible.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)

                # ws = writer.sheets[sheet_name]
                # for i, (txt, url) in enumerate(zip(df["Subject"], df["Link"]), start=startrow+1):
                #     if url:
                #         ws.write_url(i, 1, url, string=txt)

    print(f"Excel saved: {excel_out}")


# Example usage
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-23\25-09-23T15-32-09_cache.json"
cache_to_excel(path, "Orders_example_All.xlsx")


There are less than three cols, skipping
Excel saved: Orders_example_All.xlsx


In [23]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://ibbi.gov.in/en/orders/nclt?title=&date=2025-09-22&nclt=25")

# all input fields inside the form
inputs = driver.find_elements(By.CSS_SELECTOR, "#myFormId input")

for inp in inputs:
    name = inp.get_attribute("name")
    itype = inp.get_attribute("type")
    value = inp.get_attribute("value")

    print(f"Name: {name}, Type: {itype}, Value: {value}")

time.sleep(5)
driver.quit()


[WARNING] The chromedriver version (139.0.7258.138) detected in PATH at C:\Users\rando\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (140.0.7339.128); currently, chromedriver 140.0.7339.185 is recommended for chrome 140.*, so it is advised to delete the driver in PATH and retry


Name: title, Type: text, Value: 
Name: date, Type: text, Value: 2025-09-22
Name: , Type: submit, Value: Apply


In [13]:
import requests
import pandas as pd
from io import BytesIO

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

def fetch_rba():
    url = "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    return pd.read_excel(BytesIO(r.content))

def fetch_banxico():
    url = "https://www.banxico.org.mx/SieInternet/consultarDirectorioInternetAction.do?accion=consultarCuadro&idCuadro=CF101&sector=18&locale=en"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    return pd.read_html(r.text)[0]

def fetch_bcra():
    url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    r = session.get(url, timeout=15, verify=False)
    r.raise_for_status()
    return pd.read_html(r.text)[0]

def fetch_boc():
    url = "https://www.bankofcanada.ca/rates/interest-rates/canadian-interest-rates/"
    params = {
        "lookupPage": "lookup_canadian_interest.php",
        "startRange": "2015-09-23",
        "rangeType": "dates",
        "dFrom": "2025-08-01",
        "dTo": "2025-09-22",
        "rangeValue": "1",
        "rangeWeeklyValue": "1",
        "rangeMonthlyValue": "1",
        "series[]": [
            "LOOKUPS_V39079",
            "CL.CDN.MOST.1DL",
            "V39078",
            "V80691310",
            "V122530"
        ],
        "ByDate_frequency": "daily",
        "submit_button": "Submit"
    }

    session = requests.Session()
    request = requests.Request("GET", url, params=params).prepare()
    print("Requesting URL:", request.url)

    response = session.send(request, timeout=15)
    response.raise_for_status()

    response.raise_for_status()

    tables = pd.read_html(response.text)
    for i, table in enumerate(tables):
        print(f"\nTable {i} preview:")
        print(table.head())
    return tables[0] if tables else None

with pd.ExcelWriter("central_banks.xlsx") as writer:
    # fetch_rba().to_excel(writer, sheet_name="RBA", index=False)
    # fetch_banxico().to_excel(writer, sheet_name="Banxico", index=False)
    # fetch_bcra().to_excel(writer, sheet_name="BCRA", index=False)
    fetch_boc().to_excel(writer, sheet_name="BankOfCanada", index=False)
    
    
    
path ="https://www.bankofcanada.ca/rates/interest-rates/canadian-interest-rates/?lookupPage=lookup_canadian_interest.php&startRange=2015-09-23&rangeType=dates&dFrom=2025-09-18&dTo=2025-09-22&rangeValue=1&rangeWeeklyValue=1&rangeMonthlyValue=1&series%5B%5D=LOOKUPS_V39079&series%5B%5D=CL.CDN.MOST.1DL&series%5B%5D=V39078&series%5B%5D=V80691310&series%5B%5D=V122530&ByDate_frequency=daily&submit_button=Submit"

Requesting URL: https://www.bankofcanada.ca/rates/interest-rates/canadian-interest-rates/?lookupPage=lookup_canadian_interest.php&startRange=2015-09-23&rangeType=dates&dFrom=2025-08-01&dTo=2025-09-22&rangeValue=1&rangeWeeklyValue=1&rangeMonthlyValue=1&series%5B%5D=LOOKUPS_V39079&series%5B%5D=CL.CDN.MOST.1DL&series%5B%5D=V39078&series%5B%5D=V80691310&series%5B%5D=V122530&ByDate_frequency=daily&submit_button=Submit

Table 0 preview:
  V39079: Target for the overnight rate  \
0                                   Low   
1                               Average   
2                                  High   

  V39079: Target for the overnight rate.1  \
0                              2025-09-19   
1                 2025-08-01 — 2025-09-19   
2                              2025-09-17   

   V39079: Target for the overnight rate.2  
0                                     2.50  
1                                     2.74  
2                                     2.75  

Table 1 preview:
         Date

C:\Users\kaustubh.keny\AppData\Local\Temp\ipykernel_19448\3146640691.py:57: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [4]:
df = fetch_rba()

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-10\25-09-10T16-42-02_cache.json"
data = Helper.load_json(path)
OperationExecutor.generate_cache_doc_report(data)

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
path = r"25-09-10T16-42-02_cache.json"
# path = r"25-09-08T18-33-09_cache.json5"

    
root_file = os.path.basename(path).split("_")[0]
function_to_execute = {
"primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
"secondary":[["sha1","norm_table","SHA_ONE"]],
}
try:
    data = Helper.load_json(path, typ="json5")
    processed_data = ops.runner(data,function_to_execute)
    Helper.save_json(processed_data,f"{root_file}_process.json")
except Exception as e:
    print(f"\nError in Processing.. Skipping, {e}")


In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()
yesterday = Helper.load_json("25-09-09T15-14-38_process.json")
today = Helper.load_json("25-09-10T16-42-02_process.json")

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-080925.json")

ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\1109_1009_compare\compare-080925.json"
result = Helper.load_json(path)
ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'